In [ ]:
words= open('names.txt', 'r').read().splitlines()

In [ ]:
words[:10] #first 10 words

In [ ]:
len(words)

In [ ]:
min(len(w) for w in words)

In [ ]:
max(len(w) for w in words)

In [ ]:
b={}

for w in words[:3]: #here we are making tuples of adjacent words using zip
     chs=['<S>']+list(w)+['<E>'] #adding a starting and ending character
     for ch1,ch2 in zip(chs,chs[1:]):
         bigram=(ch1,ch2)
         b[bigram]=b.get(bigram,0)+1 #get the count and add 1. '0' is used as default if the tuple is not present in the dict
         

In [ ]:
b

In [ ]:
sorted(b.items(), key= lambda pair: -pair[1]) #lamda takes a variable pair(tuple:key and value) and uses the neagtive of value to sort 

In [ ]:
import torch

In [ ]:
N=torch.zeros((27,27),dtype=torch.int32)


In [ ]:
chars=sorted(list(set(''.join(words))))
stoi ={s:i+1 for i,s in enumerate(chars)}  #mapping from characters to numbers
stoi['.']=0
itos={i:s for s,i in stoi.items()} #mapping from no to characters
itos

In [ ]:
for w in words: #Now doing it for all words
     chs=['.']+list(w)+['.'] #adding a starting and ending character
     for ch1,ch2 in zip(chs,chs[1:]):
        ix1=stoi[ch1]
        ix2=stoi[ch2]
        N[ix1,ix2]+=1 #filling the matrix

In [ ]:
import matplotlib.pyplot as plt


plt.figure(figsize=(16,16))
plt.imshow(N,cmap='Blues')
for i in range(27):
    for j in range (27):
        chstr=itos[i]+itos[j]
        plt.text(j,i,chstr,ha="center",va="bottom",color='gray')
        plt.text(j,i,N[i,j].item(),ha="center",va="top",color='gray')
plt.axis('off');

### Sampling from the model

In [ ]:

N[0]


In [ ]:
p=N[0].float()
p=p/p.sum()
p

In [ ]:
g=torch.Generator().manual_seed(2147483647)
ix=torch.multinomial(p,num_samples=1,replacement=True,generator=g).item() 
itos[ix]

In [ ]:
g=torch.Generator().manual_seed(2147483647)
p=torch.rand(3,generator=g)
p=p/p.sum()
p

In [ ]:
torch.multinomial(p,num_samples=100,replacement=True,generator=g) #drawing balls from the bag with replacement when the probabilities of diff. balls is given

In [ ]:
P= (N+1).float() #adding for smoothing using add a fake count to avoid infinte loss
P/=P.sum(1,keepdim=True) #important for broadcasting

In [ ]:
g=torch.Generator().manual_seed(2147483647)
for i in range (20):
    ix=0
    out=[]
    while True:
        p=P[ix]
       # p=N[ix].float()
        #p=p/p.sum()
        ix=torch.multinomial(p,num_samples=1,replacement=True,generator=g).item()
        out.append(itos[ix])
        if ix ==0:
         break
    print(''.join(out))

In [ ]:
#Maximum Likelihood Estimation
log_likelihood=0.0
n=0
for w in words: 
     chs=['.']+list(w)+['.'] 
     for ch1,ch2 in zip(chs,chs[1:]):
        ix1=stoi[ch1]
        ix2=stoi[ch2]
        prob =P[ix1,ix2]
        logprob=torch.log(prob)
        log_likelihood+=logprob
        n+=1
#print(f'{ch1}{ch2} : {prob:.4f}')

print(f'{log_likelihood =}')
nll=-log_likelihood
print(f'{nll/n =}')

### The Neural Network Approach


In [ ]:
#creating the training set of biagrams  using just the first example
xs=[]
ys=[]
for w in words[:1]: 
     chs=['.']+list(w)+['.'] 
     for ch1,ch2 in zip(chs,chs[1:]):
        ix1=stoi[ch1]
        ix2=stoi[ch2]
        xs.append(ix1)
        ys.append(ix2)
xs=torch.tensor(xs)        
ys=torch.tensor(ys)     

In [ ]:
xs

In [ ]:
xs.shape #the input charater

In [ ]:
ys.shape

In [ ]:
ys #the most likely char given the input corresponding

In [ ]:
#one hot encoding
import torch.nn.functional as F
xenc=F.one_hot(xs,num_classes=27).float()
xenc


In [ ]:
xenc.shape


In [ ]:
plt.imshow(xenc)

In [ ]:
xenc.dtype

In [ ]:
#weights
g = torch.Generator().manual_seed(2147483647)
w=torch.rand((27,27),generator=g,requires_grad=True) # 27 neurons
xenc @ w

In [ ]:
(xenc@w).shape # 27 outputs


In [ ]:
#these 27 outputs are log(counts) i.e. logits -> exponet them

logits=(xenc@w)
#softmax
counts=(xenc@w).exp()
probs = counts/counts.sum(1,keepdims=True) #along columns
probs

In [ ]:
probs[0].sum()

In [ ]:
probs.shape

In [ ]:
# Negative log likelihoods (per example)
nlls = torch.zeros(5)

for i in range(5):
    x = xs[i].item()
    y = ys[i].item()

    print('--------')
    print(f'bigram example {i+1}: {itos[x]}{itos[y]} (indexes {x},{y})')
    print('input to the neural net:', x)
    print('output probabilities from the neural net:', probs[i])
    print('label (actual next character):', y)

    p = probs[i, y]
    print('probability assigned by the net to the correct character:', p.item())

    logp = torch.log(p)
    print('log likelihood:', logp.item())

    nll = -logp
    print('negative log likelihood:', nll.item())

    nlls[i] = nll

print('========')
print('average negative log likelihood, i.e. loss =', nlls.mean().item())

### OPtimization of the Neural Net

In [ ]:
xs

In [ ]:
ys

In [ ]:
# randomly initialize 27 neurons' weights. each neuron receives 27 inputs
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27), generator=g, requires_grad=True)

In [ ]:
# forward pass
xenc = F.one_hot(xs, num_classes=27).float() # input to the network: one-hot encoding
logits = xenc @ W # predict log-counts
counts = logits.exp() # counts, equivalent to N
probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
loss = -probs[torch.arange(5), ys].log().mean()

In [ ]:
print(loss.item())

In [ ]:
# backward pass
W.grad = None # set to zero the gradient
loss.backward()

In [ ]:
W.data += -0.1 * W.grad #updating the weights

### Entire Neural Net Code

In [ ]:
# create the dataset
xs, ys = [], []
for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    xs.append(ix1)
    ys.append(ix2)
xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
print('number of examples: ', num)

# initialize the 'network'
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27), generator=g, requires_grad=True)

In [ ]:
# gradient descent
for k in range(200):
  
  # forward pass
  xenc = F.one_hot(xs, num_classes=27).float() # input to the network: one-hot encoding
  logits = xenc @ W # predict log-counts
  counts = logits.exp() # counts, equivalent to N
  probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
  loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()
  print(loss.item())
  
  # backward pass
  W.grad = None # set to zero the gradient
  loss.backward()
  
  # update
  W.data += -50 * W.grad

In [ ]:
# Sampling from the neural net model
g = torch.Generator().manual_seed(2147483647)

for i in range(5):
  
  out = []
  ix = 0
  while True:
    
    # ----------
    # BEFORE:
    #p = P[ix]
    # ----------
    # NOW:
    xenc = F.one_hot(torch.tensor([ix]), num_classes=27).float()
    logits = xenc @ W # predict log-counts
    counts = logits.exp() # counts, equivalent to N
    p = counts / counts.sum(1, keepdims=True) # probabilities for next character
    # ----------
    
    ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    out.append(itos[ix])
    if ix == 0:
      break
  print(''.join(out))